# Spark NLP

Building a spam Detection using NLP.


In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector
from pyspark.ml.classification import NaiveBayes
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [ ]:
data = spark.read.csv("DataSets/SMSSpamCollection",inferSchema=True,sep='\t')

In [ ]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [ ]:
data.show()

## Clean and Prepare the Data

** Create a new length feature: **

In [ ]:
from pyspark.sql.functions import length

In [ ]:
data = data.withColumn('length',length(data['text']))

In [ ]:
data.show()
data.head()

In [ ]:
# Pretty Clear Difference
data.groupby('class').mean().show()

## Feature Transformations

In [ ]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer

tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")

"""
Converting the Target Variable to Numeric Labels
"""
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')



In [ ]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')


### The Model

We'll use Naive Bayes, but feel free to play around with this choice!

In [ ]:
# Use defaults
nb = NaiveBayes()

### Pipeline

In [ ]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,clean_up])

In [ ]:
cleaner = data_prep_pipe.fit(data)

In [ ]:
clean_data = cleaner.transform(data)
clean_data.head()

### Training and Evaluation!

In [ ]:
clean_data = clean_data.select(['label','features'])

In [ ]:
clean_data.show()

In [ ]:
(training,testing) = clean_data.randomSplit([0.7,0.3])

In [ ]:
spam_predictor = nb.fit(training)

In [ ]:
data.printSchema()

In [ ]:
test_results = spam_predictor.transform(testing)

In [ ]:
test_results.show()

In [ ]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting spam was: {}".format(acc))